<div style="text-align: center; font-size: 30pt; font-weight: bold; margin: 1em 0em 1em 0em">Homework 1</div>

$\textbf{Authors}$ : Adel Nabli

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from time import time
from tqdm import tqdm

L:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load the data in the format of a design matrix for X
# and one-hot encode Y

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

X_train = np.reshape(x_train, (len(x_train), 784))
X_test = np.reshape(x_test, (len(x_test), 784))
Y_train = OneHotEncoder().fit_transform(np.reshape(y_train, (len(y_train),1))).toarray()
Y_test = OneHotEncoder().fit_transform(np.reshape(y_test, (len(y_test),1))).toarray()

# Problem 1:

We want to build a MLP with two hidden layers with $h^1$ and $h^2$ hidden units. The data is given in the form of a design matrix $X \in \mathbb{R}^{n \times 784}$ and $\forall i \in [\![1,n]\!], y^i \in \mathbb{R}^{10}$ is the one-hot encoding of the label.
The output layer is parametrized by a $softmax$ function $s(z)_c = \dfrac{e^{z_c}}{\sum_c e^{z_c}}$.

* We choose the sigmoid function as a non linearity for both layers.
We have: $\sigma(z) = \dfrac{1}{1+ e^{-z}}$ and $\sigma'(z) = (1-\sigma(z))\sigma(z)$.


* Thus, our NN is computing, for each example $x \in \mathbb{R}^{784}$:

    - $A_1 = \sigma(W_1 x + b_1)$ with $W_1 \in \mathbb{R}^{h_1 \times 784}$ and $b_1 \in \mathbb{R}^{h_1}$
    - $A_2 = \sigma(W_2 A_1 + b_2)$ with $W_2 \in \mathbb{R}^{h_2 \times h_1}$ and $b_2 \in \mathbb{R}^{h_2}$
    - $O = s(W_3 A_2 + b_3)$ with $W_3 \in \mathbb{R}^{10 \times h_2}$ and $b_3 \in \mathbb{R}^{10}$


* The cross entropy loss is defined by: $l(y, O) = -\sum_{c=1}^{10} y_c\log(o_c)$ (recall that $y_c = \mathbb{1}_{label=c}$) which leads to an empirical risk $\hat{R} = \dfrac{-1}{n} \sum_{i=1}^n \sum_{c=1}^{10} y_c^i\log(o_c^i)$.

* Our purpose is to learn each parameter $\theta \in \{ W_1, b_1, W_2, b_2, W_3, b_3 \}$ by **stochastic** gradient descent incrementaly: $\theta^{t+1} = \theta^t - \alpha_t \dfrac{\partial l}{\partial \theta}(x_t, y_t)$  with $\alpha_t, x_t, y_t$ being respectively the learning rate, the training example and the corresponding label at time $t$. We use the formula $\alpha_t = \dfrac{\alpha_0}{1 + \delta t}$ for the learning rate.


* For that, using the chain-rule, we derive:

    - $\dfrac{\partial l}{\partial b_3} = O - y$
    - $\dfrac{\partial l}{\partial W_3} = (O - y)A_2^{T}$
    - $\dfrac{\partial l}{\partial b_2} = \dfrac{\partial l}{\partial A_2}*(1-A_2)*A_2$ with $\dfrac{\partial l}{\partial A_2} = W_3 ^T  \dfrac{\partial l}{\partial b_3}$
    - $\dfrac{\partial l}{\partial W_2} = \dfrac{\partial l}{\partial b_2 } A_1 ^T$
    - $\dfrac{\partial l}{\partial b_1} = \dfrac{\partial l}{\partial A_1}*(1-A_1)*A_1$ with $\dfrac{\partial l}{\partial A_1} = W_2 ^T  \dfrac{\partial l}{\partial b_2}$
    - $\dfrac{\partial l}{\partial W_1} = \dfrac{\partial l}{\partial b_1 } x ^T$


* To initialize our parameters, we have three options:
    - **Zero**: we initialize with zeros
    - **Normal**: we initialize with $\mathcal{N}(0,1)$
    - **Glorot**: we initialize with $\mathcal{U}(-d^l, d^l)$ where $d^l = \sqrt{\dfrac{6}{h^{l-1}+h^l}}$

In [4]:
class NN(object):
    
    def __init__(self, hidden_dims=(1024,2048), initialization='glorot', learning_rate=0.001, delta=0.7):
        
        dims = (784, hidden_dims[0], hidden_dims[1], 10)
        W1, b1, W2, b2, W3, b3 = self.initialize_weights(dims, initialization)
        parameters = dict()
        parameters['W1'] = W1
        parameters['b1'] = b1
        parameters['W2'] = W2
        parameters['b2'] = b2
        parameters['W3'] = W3
        parameters['b3'] = b3
        
        cache = dict()
        cache['x'] = None
        cache['A1'] = None
        cache['A2'] = None
        cache['O'] = None
        cache['y'] = None
        
        grad = dict()
        grad['W1'] = None
        grad['b1'] = None
        grad['W2'] = None
        grad['b2'] = None
        grad['W3'] = None
        grad['b3'] = None
        
        self.parameters = parameters
        self.cache = cache
        self.grad = grad
        self.learning_rate = learning_rate
        self.delta = delta
        self.cpt = 0
        self.n_samples = 0
        
    def initialize_weights(self,dims, initialization):
        
        # dims = (dim_input, h1, h2, dim_output)
        
        np.random.seed(42)
        
        d, h1, h2, o = dims
        b1 = np.zeros(h1)
        b2 = np.zeros(h2)
        b3 = np.zeros(o)
        
        if initialization == 'zero':
            
            W1 = np.zeros((h1, d))
            W2 = np.zeros((h2,h1))
            W3 = np.zeros((o,h2))
        
        elif initialization == 'normal':
            
            W1 = np.random.normal(0,1,(h1,d))
            W2 = np.random.normal(0,1,(h2,h1))
            W3 = np.random.normal(0,1,(o,h2))
        
        elif initialization == 'glorot':
            
            d1 = np.sqrt(6/(d+h1))
            d2 = np.sqrt(6/(h1+h2))
            d3 = np.sqrt(6/(h2+o))
            
            W1 = np.random.uniform(-d1,d1, (h1,d))
            W2 = np.random.uniform(-d2,d2, (h2,h1))
            W3 = np.random.uniform(-d3,d3, (o,h2))
        
        return(W1, b1, W2, b2, W3, b3)

    def forward(self,x,y):
        
        W1 = self.parameters['W1']
        b1 = self.parameters['b1']
        W2 = self.parameters['W2']
        b2 = self.parameters['b2']
        W3 = self.parameters['W3']
        b3 = self.parameters['b3']
        
        A1 = self.activation(np.dot(W1,x)+b1)
        A2 = self.activation(np.dot(W2,A1)+b2)
        O = self.softmax(np.dot(W3,A2)+b3)
        
        self.cache['x'] = x
        self.cache['A1'] = A1
        self.cache['A2'] = A2
        self.cache['O'] = O
        self.cache['y'] = y

    def activation(self,U):
        
        return(1/(1+np.exp(-U)))

    def loss(self, O, y_true):
        
        return(-np.log(np.dot(O,y_true)))
        
    def softmax(self,U):
        
        return(np.exp(U - np.log(np.sum(np.exp(U), axis=0))))

    def backward(self):
        
        x = self.cache['x']
        A1 = self.cache['A1']
        A2 = self.cache['A2']
        O = self.cache['O']
        y = self.cache['y']
        W3 = self.parameters['W3']
        W2 = self.parameters['W2']
        
        db3 = O-y
        self.grad['b3'] = db3
        self.grad['W3'] = np.dot(np.reshape(db3, (len(db3),1)), np.reshape(A2, (1, len(A2))))
        dA2 = np.dot(W3.T,db3)
        self.grad['b2'] = dA2*(1-A2)*A2
        db2 = self.grad['b2']
        self.grad['W2'] = np.dot(np.reshape(db2, (len(db2),1)), np.reshape(A1, (1, len(A1))))
        dA1 = np.dot(W2.T,db2)
        self.grad['b1'] = dA1*(1-A1)*A1
        db1 = self.grad['b1']
        self.grad['W1'] = np.dot(np.reshape(db1, (len(db1),1)), np.reshape(x, (1, len(x))))

    def update(self):
        
        alpha_0 = self.learning_rate
        delta = self.delta
        self.cpt += 1
        
        alpha = alpha_0/(1+delta*self.cpt)
        
        self.parameters['W1'] -= alpha*self.grad['W1']
        self.parameters['b1'] -= alpha*self.grad['b1']
        self.parameters['W2'] -= alpha*self.grad['W2']
        self.parameters['b2'] -= alpha*self.grad['b2']
        self.parameters['W3'] -= alpha*self.grad['W3']
        self.parameters['b3'] -= alpha*self.grad['b3']

    def train(self, X_train, Y_train, X_test, Y_test, n_epoch):
        
        n = len(X_train)
        self.n_samples = n
        ids = np.arange(n)
        averaged_loss_train = []
        averaged_loss_test = []
        accuracy_train = []
        accuracy_test = []
        
        for epoch in range(n_epoch):
            
            t0 = time()
            
            print('Epoch : %s' %epoch, '\n')
            
            np.random.shuffle(ids)
            risk = 0
            
            for i in tqdm(range(n)):
                
                x = X_train[ids[i]]
                y = Y_train[ids[i]]
                
                self.forward(x,y)
                self.backward()
                self.update()
                
                risk += self.loss(self.cache['O'], y)
            
            risk = risk/n
            averaged_loss_train.append(risk)
            Y_predict_test = self.predict(X_test.T)
            averaged_loss_test.append(np.mean(np.diag(self.loss(Y_predict_test.T, Y_test.T))))
            
            Y_predict_train = self.predict(X_train.T)
            y_predict_train = np.argmax(Y_predict_train, axis=0)
            y_true_train = np.where(Y_train==1)[1]
            y_predict_test = np.argmax(Y_predict_test, axis=0)
            y_true_test = np.where(Y_test==1)[1]
            accuracy_train.append(accuracy_score(y_true_train, y_predict_train))
            accuracy_test.append(accuracy_score(y_true_test, y_predict_test))
            
            t1 = time() - t0
            print('Training loss = %s' %risk, '\n')
            print('Time taken = %s' %t1, ' seconds' '\n')
            print('========================', '\n')
        
        fig = plt.figure(figsize=(16,8))
        ax1 = fig.add_subplot(221)
        ax1.plot(np.arange(n_epoch), averaged_loss_train, label='training loss')
        ax1.plot(np.arange(n_epoch), averaged_loss_test, label='test loss')
        ax1.set_title('Evolution of the averaged loss during training')
        ax1.legend()
        ax2 = fig.add_subplot(222)
        ax2.plot(np.arange(n_epoch), accuracy_train, label='accuracy train')
        ax2.plot(np.arange(n_epoch), accuracy_test, label='accuracy test')
        ax2.legend()
        ax2.set_title('Evolution of the accuracy during the training')
        
        plt.show()
    
    def predict(self, x):
        
        W1 = self.parameters['W1']
        b1 = self.parameters['b1']
        W2 = self.parameters['W2']
        b2 = self.parameters['b2']
        W3 = self.parameters['W3']
        b3 = self.parameters['b3']
        
        A1 = self.activation(np.dot(W1,x)+np.reshape(b1, (len(b1),1)))
        A2 = self.activation(np.dot(W2,A1)+np.reshape(b2, (len(b2),1)))
        O = self.softmax(np.dot(W3,A2)+np.reshape(b3, (len(b3),1)))
        
        return(O)

In [ ]:
MLP = NN(hidden_dims=(400,700), initialization='glorot', learning_rate=0.003, delta=2e-5)
MLP.train(X_train, Y_train, X_test, Y_test, 15)